In [1]:
from minio import Minio
from minio.error import ResponseError
from urllib3.exceptions import MaxRetryError
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError
import numpy as np
import tempfile

from pyspark import SparkContext, SparkConf

## Configuration la connection au Minio et Elastic Search

In [2]:
bucketName = 'train'
bucketName2 = 'test'
indexName = 'images'

minioClient = Minio('localhost:9001', access_key='minio', secret_key='minio123', secure=False)
es = Elasticsearch([{'host':'localhost','port':9200}])

## Chercher dans Elastic Search

In [3]:
key_word = 'forest'
directory = 'test'

In [4]:
result = es.search(index=directory, q='predict_label:' + key_word, filter_path='hits.hits._source',size=10)

## Afficher images et ses attribut

In [5]:
records = result['hits']['hits']

In [6]:
records

[{'_source': {'url': 's3a://test/test-img7.npy',
   'name': 'test-img7.npy',
   'test_label': 'forest',
   'predict_label': 'forest',
   'test_array': [0.0, 0.0, 1.0, 0.0, 0.0],
   'predict_array': [0.00601671077311039,
    0.047392837703228,
    0.5142027735710144,
    0.06994199752807617,
    0.013775604777038097]}},
 {'_source': {'url': 's3a://test/test-img11.npy',
   'name': 'test-img11.npy',
   'test_label': 'agricultural_territory',
   'predict_label': 'forest',
   'test_array': [0.0, 1.0, 0.0, 0.0, 0.0],
   'predict_array': [0.06451502442359924,
    0.39509111642837524,
    0.4295840561389923,
    0.090579092502594,
    0.04068891331553459]}},
 {'_source': {'url': 's3a://test/test-img8.npy',
   'name': 'test-img8.npy',
   'test_label': 'forest',
   'predict_label': 'forest',
   'test_array': [0.0, 0.0, 1.0, 0.0, 0.0],
   'predict_array': [0.0024664762895554304,
    0.030774887651205063,
    0.569710910320282,
    0.043337561190128326,
    0.006072014570236206]}},
 {'_source': {'

In [7]:
print(records[0]['_source']['name'])

test-img7.npy


In [8]:
for r in records:
    print(minioClient.fget_object(directory, r['_source']['name'], '/tmp/'+r['_source']['name']))

<Object: bucket_name: test object_name: b'test-img7.npy' last_modified: time.struct_time(tm_year=2019, tm_mon=1, tm_mday=24, tm_hour=20, tm_min=11, tm_sec=32, tm_wday=3, tm_yday=24, tm_isdst=0) etag: 357ecd4d0339e6ccf18cfaaf8c5ec85e size: 24704 content_type: application/octet-stream, is_dir: False, metadata: {}>
<Object: bucket_name: test object_name: b'test-img11.npy' last_modified: time.struct_time(tm_year=2019, tm_mon=1, tm_mday=24, tm_hour=20, tm_min=11, tm_sec=32, tm_wday=3, tm_yday=24, tm_isdst=0) etag: ea29d9c5942e9171883f7958d4f81980 size: 24704 content_type: application/octet-stream, is_dir: False, metadata: {}>
<Object: bucket_name: test object_name: b'test-img8.npy' last_modified: time.struct_time(tm_year=2019, tm_mon=1, tm_mday=24, tm_hour=20, tm_min=11, tm_sec=32, tm_wday=3, tm_yday=24, tm_isdst=0) etag: 4f1fccddb2d12ba882c3d3f8ad91ed32 size: 24704 content_type: application/octet-stream, is_dir: False, metadata: {}>
<Object: bucket_name: test object_name: b'test-img10.npy'

In [ ]:
conf = SparkConf()
sc = SparkContext(conf=conf)

b1 = sc.textFile('s3a://test/test-img6.npy')
b1.collect()